## Import Libraries

In [1]:

import os
import json
import requests
import itertools
import numpy as np
import pandas as pd
from pprint import pprint
from sqlalchemy.orm import Session
from sqlalchemy import create_engine


## Connect to Postgress SQL DB

In [2]:

pg_user = 'postgres'
pg_password = 'password'
db_name = 'Enterprises'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')



#### Get table Names

In [3]:

engine.table_names()


['NAICS', 'STATES', 'BUSINESSES']

#### Create dataframes from SQL Query

In [4]:

bus_df = pd.read_sql_query('select * from "BUSINESSES"', con=engine)
naics_df = pd.read_sql_query('select * from "NAICS"', con=engine)
states_df = pd.read_sql_query('select * from "STATES"', con=engine)


#### Refine Dataframesto only include relevant columns

In [5]:

business_df = bus_df[[ 'STATE DESCRIPTION', 'NAICS CODE', 'ENTERPRISE EMPLOYMENT SIZE 2', 'NUMBER OF FIRMS', 'NUMBER OF ESTABLISHMENTS', 'EMPLOYMENT', 'ANNUAL PAYROLL ($1,000)', 'YEAR']].copy()
business_df = business_df.rename(columns = {'ENTERPRISE EMPLOYMENT SIZE 2': 'ENTERPRISE EMPLOYMENT SIZE' })

states_df = states_df.rename(columns={'STATE':'STATE DESCRIPTION'})
states_df = states_df[["STATE DESCRIPTION","STATE CODE"]]


#### Convert Payroll column to real value from rounded value

In [6]:

business_df['ANNUAL PAYROLL'] = business_df['ANNUAL PAYROLL ($1,000)'] * 1000


#### Delete Original Payroll Column

In [7]:

del business_df['ANNUAL PAYROLL ($1,000)']


#### Create joins to establish consolidated dataframe

In [8]:

business_df1 = pd.merge(business_df,naics_df,on='NAICS CODE',how='left')
business_df1= pd.merge(business_df1,states_df,on='STATE DESCRIPTION',how='left')
business_df1.head()


,STATE DESCRIPTION,NAICS CODE,ENTERPRISE EMPLOYMENT SIZE,NUMBER OF FIRMS,NUMBER OF ESTABLISHMENTS,EMPLOYMENT,YEAR,ANNUAL PAYROLL,NAME,STATE CODE
0,Alabama,--,Total,79812,103875,1714692,2008,59827325000,Total,AL
1,Alabama,--,0-4,43194,43259,80104,2008,2419258000,Total,AL
2,Alabama,--,5-9,15160,15349,100066,2008,2791861000,Total,AL
3,Alabama,--,10-19,9107,9588,121129,2008,3623405000,Total,AL
4,Alabama,--,<20,67461,68196,301299,2008,8834524000,Total,AL


#### Reorder Dataframe

In [9]:

business_df1 = business_df1[['YEAR','STATE DESCRIPTION', 'STATE CODE', 'NAICS CODE', 'NAME', 'ENTERPRISE EMPLOYMENT SIZE', 'NUMBER OF FIRMS', 'NUMBER OF ESTABLISHMENTS', 'EMPLOYMENT', 'ANNUAL PAYROLL']].copy()
business_df1.head()


,YEAR,STATE DESCRIPTION,STATE CODE,NAICS CODE,NAME,ENTERPRISE EMPLOYMENT SIZE,NUMBER OF FIRMS,NUMBER OF ESTABLISHMENTS,EMPLOYMENT,ANNUAL PAYROLL
0,2008,Alabama,AL,--,Total,Total,79812,103875,1714692,59827325000
1,2008,Alabama,AL,--,Total,0-4,43194,43259,80104,2419258000
2,2008,Alabama,AL,--,Total,5-9,15160,15349,100066,2791861000
3,2008,Alabama,AL,--,Total,10-19,9107,9588,121129,3623405000
4,2008,Alabama,AL,--,Total,<20,67461,68196,301299,8834524000


#### Table with all relavent data to be jsonified

In [10]:

table_df = business_df1.rename(columns = {'STATE DESCRIPTION': 'STATE_DESCRIPTION', 'STATE CODE' : 'STATE_CODE', 'NAICS CODE' : 'NAICS_CODE' , 'ENTERPRISE EMPLOYMENT SIZE' : 'ENTERPRISE_EMPLOYMENT_SIZE', 'NUMBER OF FIRMS': 'NUMBER_OF_FIRMS', 'NUMBER OF ESTABLISHMENTS' : 'NUMBER_OF_ESTABLISHMENTS', 'ANNUAL PAYROLL' : 'ANNUAL_PAYROLL' })
table_df.head()


,YEAR,STATE_DESCRIPTION,STATE_CODE,NAICS_CODE,NAME,ENTERPRISE_EMPLOYMENT_SIZE,NUMBER_OF_FIRMS,NUMBER_OF_ESTABLISHMENTS,EMPLOYMENT,ANNUAL_PAYROLL
0,2008,Alabama,AL,--,Total,Total,79812,103875,1714692,59827325000
1,2008,Alabama,AL,--,Total,0-4,43194,43259,80104,2419258000
2,2008,Alabama,AL,--,Total,5-9,15160,15349,100066,2791861000
3,2008,Alabama,AL,--,Total,10-19,9107,9588,121129,3623405000
4,2008,Alabama,AL,--,Total,<20,67461,68196,301299,8834524000


In [11]:

table_df['ENTERPRISE_EMPLOYMENT_SIZE'].unique()


array([' Total', ' 0-4', ' 5-9', ' 10-19', ' <20', ' 20-99', ' 100-499',
       ' <500', ' 500+', '<500', '500+', '0-4', '5-9', '10-19', '<20',
       '20-99', '100-499', 'Total', '<5'], dtype=object)

In [12]:

table_df['ENTERPRISE_EMPLOYMENT_SIZE'].unique()

good_data = ['0-4','5-9', '10-19', '20-99','100-499','500+']

bad_data = ['Total',' <20','<500' ,'<20','<5']

table_df['ENTERPRISE_EMPLOYMENT_SIZE'] = table_df['ENTERPRISE_EMPLOYMENT_SIZE'].astype(str).str.strip()


In [13]:

table_df = table_df.loc[~table_df['ENTERPRISE_EMPLOYMENT_SIZE'].isin(bad_data)]


In [14]:

table_df['ENTERPRISE_EMPLOYMENT_SIZE'].unique()


array(['0-4', '5-9', '10-19', '20-99', '100-499', '500+'], dtype=object)

In [15]:

#Export to csv
output_file = os.path.join("static","data","table_df.csv")
table_df.to_csv(output_file, index=False, header=True)
